#### This notebook contains the code for inferencing on [ILC](https://huggingface.co/datasets/d0r1h/ILC) testset using model [led-base-ilc](https://huggingface.co/d0r1h/led-base-ilc)

Author: [Pawan Trivedi](https://twitter.com/d0r1h) <br>
Date created: 2022/05/06 <br>
Last modified: 2022/05/06 <br>
Description: Inference on test set for summarization task

In [1]:
!pip install transformers datasets sentencepiece rouge -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
import pandas as pd
from rouge import Rouge
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [5]:
rouge = Rouge()
dataset = load_dataset("d0r1h/ILC", split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/368 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/36.5M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2058 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1015 [00:00<?, ? examples/s]

In [6]:
CasesText = dataset['Case']
GoldSummary = dataset['Summary']

In [7]:
len(CasesText), len(GoldSummary)

(1015, 1015)

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
def summarize(model, tokenizer, Cases):

  SystemSummaries = []
  for i, case in enumerate(Cases):

      input_ids = tokenizer(case, return_tensors="pt").input_ids.to(device)
      global_attention_mask = torch.zeros_like(input_ids)
      global_attention_mask[:, 0] = 1
      sequences = model.generate(input_ids, global_attention_mask=global_attention_mask)
      Summary = tokenizer.batch_decode(sequences, skip_special_tokens=True)

      SystemSummaries.append(Summary)
      print(i)

  return SystemSummaries

In [10]:
checkpoint = "d0r1h/led-base-ilc"

tokenizer_led = AutoTokenizer.from_pretrained(checkpoint)
model_led = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

In [11]:
SystemSummary = summarize(model_led, tokenizer_led, CasesText)

Input ids are automatically padded from 2988 to 3072 to be a multiple of `config.attention_window`: 1024


KeyboardInterrupt: 

In [12]:
SystemSummaryFinal = []

for i in SystemSummary:
  SystemSummaryFinal.append((i[0]))

NameError: name 'SystemSummary' is not defined

In [ ]:
Summaries = pd.DataFrame(list(zip(GoldSummary, SystemSummaryFinal)), columns =['GoldSummary', 'SystemSummary'])

We have inference on test datset in batches due to the time limit on google colab

In [ ]:
dir_path = "/content/drive/MyDrive/Working | Project/ILC/data/"

In [ ]:
df1 = pd.read_csv(dir_path + "Summaries1.csv")
df2 = pd.read_csv(dir_path + "Summaries2.csv")
df3 = pd.read_csv(dir_path + "Summaries3.csv")
df4 = pd.read_csv(dir_path + "Summaries4.csv")

In [ ]:
DF = pd.concat([df1, df2, df3, df4])
DF.reset_index(inplace=True, drop=True)

In [ ]:
DF

,GoldSummary,SystemSummary
0,Section 86(1)(f) vests a statutory jurisdictio...,In the case of Gujarat Urja Vikas Nigam Limite...
1,The petitioner apprehended arrest under Sectio...,The case was taken up out of turn on the basis...
2,The petitioner was arrested under Sections 344...,The petitioner is running Dance Party from man...
3,In matters concerning administrative appointme...,The petitioners were appointed as Inspectors i...
4,The facts and information of the suspected off...,The FIR was registered on the basis of a repor...
...,...,...
1010,"In the present case, an appeal is preferred un...",The court observed that the court has not inve...
1011,Re-evaluation of answer sheets is not permissi...,“If an error is committed by the examination a...
1012,The presence of an alternate land that can be ...,The Landlord has a registered Society under th...
1013,Bail may be granted to an individual who is ac...,"The applicant is arrested on 18th November, 20..."


In [ ]:
score = rouge.get_scores(DF['SystemSummary'], DF['GoldSummary'], avg=True)

In [ ]:
LEDRouge = pd.DataFrame(score).set_index([['recaall','precision','f-measure']])

In [ ]:
LEDRouge*100

,rouge-1,rouge-2,rouge-l
recaall,40.039816,22.343800,37.250347
precision,46.313938,25.252847,43.097858
f-measure,42.240362,23.187177,39.304978


In [ ]:
DF.to_csv(dir_path + "LEDPrediction.csv", index=False, header=True)

In [ ]:
LEDRouge.to_csv("/content/drive/MyDrive/Working | Project/ILC/score/LEDRouge.csv", header=True)